In [3]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy
import matplotlib.cm as cm
import matplotlib.colorbar as cb


In [ ]:
import re

prMatcher=re.compile('ps')
prPrecMatcher=re.compile('Rain')
tsMatcher=re.compile('Temp')


In [25]:
# A function to trim to the area of interest for this study
def domain(ds):
    return ds.where(
    (ds.lat<-28)
        *(ds.lat>-39)
        *(ds.lon>144)
        *(ds.lon<155),
    drop=True
)
    
    

# Some functions

This function plots an contour map anomaly using a provided xarray object and an axis.

In [1]:
def plotAnom(da, ax, showOcean=True):
    """For a provided dataarray and axis, plot the data, add coastlines and """
    
    import cartopy.feature as cfeature
    import matplotlib.ticker as mticker
    
        #set contour intervals and colours
    if prPrecMatcher.search(da.name):
        precContours=numpy.arange(-50,51,10)
        cmap='BrBG'
    elif prMatcher.search(da.name):
        precContours=numpy.arange(-1.6,1.61,0.2)
        cmap='BrBG'
    elif tsMatcher.search(da.name):
        precContours=numpy.arange(-2,2,0.25)
        cmap='coolwarm'
    else:
        precContours=numpy.linspace(da.min(), da.max(), 7)
        cmap='viridis'
    
     #do the plotting
    cs=plt.contourf(
        da.lon, da.lat, da , precContours, 
        transform=ccrs.PlateCarree(), cmap=cmap, extend='both'
    )
    
    #draw the coast too
    ax.coastlines(color='black')
    if showOcean==False:
        ax.add_feature(cfeature.OCEAN,zorder=2,color='white')
        
    #deaw a box around the area of interest
    ax.add_patch(
        mpatches.Rectangle(
            xy=[144, -39], width=11, height=11,edgecolor='red',facecolor='none',zorder=3
        ),
    )
        
    gl=ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle=':',zorder=3)
    gl.top_labels=False
    gl.left_labels=False
    gl.xlocator = mticker.FixedLocator([120, 130, 140, 150, 160])
    gl.ylocator = mticker.FixedLocator([-50,-40,-30,-20,-10,0])
    

    return cs , gl


In [30]:
def plotStipling(da, ax):
    """For a provided Dataarray and axis, 
    add stipling where less than 70% of models agree with the sign of the change.
    
    Provided Dataarray is assumed to:
    - have dimensions lon,lat and model
    - be a datarray representing the mean impact in each model from 0/1/2/3 climate modes
    """
    
    try:
        if len(da.model.values)<2:
            raise EnvironmentError("Nothing to stiple, there needs to be at least 2 models")
    except:
        raise EnvironmentError("Does this dataarray have a model dimension?")
    #Error checking should probably check there are only three dimensions.
    
    criteria=0.7*len(da.model.values) #70% of models

    #Find areas where the mean change is less than 0
    meanLessThan=da.mean('model')<0
    #Find how many models agree on the sign of the change
    lessThanCount=(da<0).sum('model')
    #Find areas where the number of models is less than the criteria (70%) 
    condLessThan=lessThanCount<criteria

    #Repeat for greater than
    meanGreatThan=da.mean('model')>0
    greatThanCount=(da>0).sum('model')
    condGreatThan=greatThanCount<criteria
    
    #If mean is 0, this would mean no data, so do nothing.

    #Area to add stipling 
    combined=(
        (meanLessThan&condLessThan) | (meanGreatThan&condGreatThan)
    )

    #plot it
    plt.sca(ax)
    plt.pcolor(
        combined.lon, 
        combined.lat,
        numpy.ma.masked_equal(combined,0),
        hatch='.',
        alpha=0,
        shading='auto'
    )



This function loops though all the provides indeces and plots them for the date ranges 850-1850, 1850-2005 and 2005-2100

In [1]:
def multiPlotter(da, mask, indeces, stipling=True, cbarLabel='_nolabel_'):

    #Decide how many rows we are going to have
    if da.year[0]<1850:
        years=[
            [850,1850],[1950,1999],[2050,2099]
        ]
        lenYears=3
    elif da.year[-1]<2050:
        years=[[1900,2000]]
        lenYears=1
    else:
        years=[
            [1950,1999],
            [2050,2099]
        ]
        lenYears=2

    #how many columns
    lenIndeces=len(indeces)
    
    fig = plt.figure(figsize=(3*lenIndeces,3*(lenYears+1)))
    
    axs=list()
    
    #for each row
    for iGroup in range(0,lenYears):

        iYears=years[iGroup]
        
        #for each column
        for i in range(0,lenIndeces):
            
            #make a sub plot
            ax=fig.add_subplot(
                lenYears+1, #how many rows
                lenIndeces, #how many columns
                i+1+iGroup*(lenIndeces), # index
                projection=ccrs.PlateCarree()
            )
              #years to plot
            plotCond = mask[indeces[i]].sel(year=slice(iYears[0],iYears[1]))
            
            toPlot = da.where(plotCond).mean(['year'])
            
            cs, gl = plotAnom(toPlot.mean('model'), ax)
            
            if stipling:
                plotStipling(
                    toPlot,
                    ax
                )
            
            plt.xlim([136,163])
            plt.ylim([-47,-20])
            
            #This is the number of events. Because dataset is well-curated, we can use the count from one position to be the count for everything
            nEvents=(da.where(plotCond).isel(lon=17,lat=15).count()/(50*37)*100).values
            ax.text(
                0.035,0.045,
                "n="+f'{nEvents:.0f}'+'%', 
                transform=ax.transAxes, 
                backgroundcolor='white',
                zorder=2
            )
                            
            #in the first column, put a y-label
            if i==0:
                ax.text(-0.1, 0.5,  #xy relative to bottom left of axis
                        str(iYears[0])+'-'+str(iYears[1]), #yearRange
                        rotation='vertical', transform=ax.transAxes, 
                        verticalalignment='center',fontsize=14)
            
            #in the first row put a heading
            if iGroup==0 :
                ax.set_title(indeces[i])
                
            axs.append(ax)

    plt.subplots_adjust(hspace=0.02,wspace=0)
    
    #put a color bar on
       
    axs.append(fig.add_axes([0.3,0.3,0.4,0.03]))
    cbar=plt.colorbar(
        cm.ScalarMappable(cs.norm,cs.cmap),
        orientation='horizontal', 
        cax=axs[-1], 
        extend='both', 
        boundaries=cs.get_array(), 
        label = cbarLabel,
    )
    
    return axs
    



In [1]:
def indexPlotter(impactXr, fEventsXr, fEventsLs, showOcean=True, stipling=True, cbarLabel='_nolabel_'):
    
    axs=list()
    nIndex=len(fEventsLs)

    dimsNotLatLon=list(impactXr.dims)
    dimsNotLatLon.remove('lat')
    dimsNotLatLon.remove('lon')

    
    fig=plt.figure(figsize=(3*nIndex,6))


    for i in range(0,nIndex):

        iEvent = fEventsLs[i]

        ax=fig.add_subplot(1, nIndex, i+1, projection=ccrs.PlateCarree())

        nEvents=impactXr.where(fEventsXr[iEvent]==True).isel(lon=15,lat=15).count().values
                            
        cs,gl = plotAnom(
            impactXr.where(fEventsXr[iEvent]==True).mean(dimsNotLatLon),
            ax,
            showOcean
        )
        
        if stipling:
            plotStipling(
                impactXr.where(fEventsXr[iEvent]==True).mean('year'),
                ax
            )
            
        plt.xlim([136,158])
        plt.ylim([-42,-20])
        
        ax.text(
            0.04,0.05,
            "n="+str(nEvents), 
            transform=ax.transAxes, 
            backgroundcolor='white',
            zorder=4
        )
        
        plt.title(iEvent)
        
        axs.append(ax)
        
        
    plt.subplots_adjust(hspace=0.02,wspace=0)

        #put a color bar on
       
    axs.append(fig.add_axes([(1-1/nIndex)/2,0.2,1/nIndex,0.03]))
    cbar=plt.colorbar(
        cm.ScalarMappable(cs.norm,cs.cmap),
        orientation='horizontal', 
        cax=axs[-1], 
        extend='both', 
        boundaries=cs.get_array(), 
        label = cbarLabel,
    )
    
        
    return axs 
        
        